# Bayes on SPY Price

## Example 1:

`Hypothesis`: SPY stock price goes up in one week

`Evidence`: price ranking in 500 days higher than 50%


$P(H|E) = \dfrac{P(E|H)\ P(H)}{P(E)}$


In [1]:
import random
random.seed(999)

In [2]:
SYMBOL = 'SPY'
WINDOW = 365
FUTURE = 5
THRESHOLD = 0.9
SAMPLE_SIZE = 500


In [3]:
from btbox import import_yahoo_csv
from pandas import Series, DataFrame
from random import randint

In [4]:
ohlcv = import_yahoo_csv(f'../../_data_/{SYMBOL}_bar1day.csv')

In [5]:
def sampling(sr: Series, draw: int) -> tuple[bool, bool]:
    for _ in range(draw):
        j = randint(WINDOW, len(sr) - 1 - FUTURE)
        win: Series = sr.iloc[j - WINDOW:j]
        future = sr.iloc[j:j + FUTURE]
        future_price_chg = future[-1] / future[0] - 1
        hypothesis = future_price_chg > 0
        current_price_ranking_pct = win.rank(pct=True).iloc[-1]
        evidence = current_price_ranking_pct > THRESHOLD
        yield (hypothesis, evidence)


samples1 = DataFrame([s for s in sampling(ohlcv.Close, SAMPLE_SIZE)],
                     columns=['Hypothesis', 'Evidence'])
samples2 = DataFrame([s for s in sampling(ohlcv.Close, SAMPLE_SIZE)],
                     columns=['Hypothesis', 'Evidence'])


In [6]:
p_H = samples1.Hypothesis.sum() / len(samples1)
p_E = samples1.Evidence.sum() / len(samples1)
samples_H = samples1[samples1.Hypothesis].Evidence
p_E_given_H = samples_H.sum() / len(samples_H)

samples_E = samples2[samples2.Evidence].Hypothesis
p_H_given_E = samples_E.sum() / len(samples_E)


In [7]:
dict(prior=p_H, posteriors=p_E_given_H * p_H / p_E)


{'prior': 0.584, 'posteriors': 0.5551601423487544}

In [8]:
dict(likelihood = p_E_given_H, bayes_factor = p_E_given_H / p_E)

{'likelihood': 0.5342465753424658, 'bayes_factor': 0.9506166821040315}

In [9]:
dict(actual=p_H_given_E)

{'actual': 0.613013698630137}